<a href="https://colab.research.google.com/github/Bint7bara/GlobusSDK/blob/main/list_tranfer_files_V4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import globus_sdk
from globus_sdk.scopes import TransferScopes

Ask for NETID at the beginning

In [ ]:
netid = input("Please enter your NETID: ").strip()

Endpoint aliases and their corresponding UUIDs and default paths

In [ ]:
endpoint_aliases = {
    "1": ("UIC ACER HPC Local Filesystems", '3eb0badf-8554-4fa4-b6a5-aa72246b4953', f'/home/{netid}/'),
    "2": ("UIC ACER Research Data Lake", 'ec0cad56-aee7-436b-b231-9cf4ec1aa2af', f'/{netid}/'),
    "3": ("UIC ACER Research Data Lake HA", 'e7599234-21f6-4ef2-aa79-f3a2b67f3b75', f'/{netid}/'),
    "4": ("UIC ACER Research Data Rapids", '4054d9ee-9045-42fa-9679-45a5dd79b220', f'/home/{netid}/.')
}

In [ ]:
def choose_endpoint():
    print("Please choose an endpoint by entering the corresponding number:")
    for key, (alias, _, _) in endpoint_aliases.items():
        print(f"{key}: {alias}")
    choice = input("Your choice: ")
    # Validate the user choice
    if choice in endpoint_aliases:
        # Replace 'NETID' with the actual netid in the path
        alias, uuid, path = endpoint_aliases[choice]
        return alias, uuid, path
    else:
        print("Invalid choice. Please try again.")
        return choose_endpoint()

In [ ]:
CLIENT_ID = "0e364f2a-14b4-472c-8aab-3d0e65c30a57"
auth_client = globus_sdk.NativeAppAuthClient(CLIENT_ID)

In [ ]:
def login_and_get_transfer_client(*, scopes=TransferScopes.all):
    auth_client.oauth2_start_flow(requested_scopes=scopes)
    authorize_url = auth_client.oauth2_get_authorize_url()
    print(f"Please go to this URL and login:\n\n{authorize_url}\n")
    auth_code = input("Please enter the code here: ").strip()
    tokens = auth_client.oauth2_exchange_code_for_tokens(auth_code)
    transfer_tokens = tokens.by_resource_server["transfer.api.globus.org"]
    return globus_sdk.TransferClient(
        authorizer=globus_sdk.AccessTokenAuthorizer(transfer_tokens["access_token"])
    )

In [ ]:
def list_files(transfer_client, endpoint_id, path):
    print(f"Listing contents of {endpoint_id} at {path}")
    try:
        for entry in transfer_client.operation_ls(endpoint_id, path=path):
            print(f"- {entry['name']}")
    except globus_sdk.TransferAPIError as e:
        print(f"An error occurred while listing files: {e}")

In [ ]:
transfer_client = login_and_get_transfer_client()

Choose source and destination endpoints

In [ ]:
print("Source Endpoint:")
src_alias, src_uuid, src_path = choose_endpoint()
print("Destination Endpoint:")
dst_alias, dst_uuid, dst_path = choose_endpoint()

List files at source and destination based on user choice

In [ ]:
print(f"\n{src_alias} Contents:")
list_files(transfer_client, src_uuid, src_path)

In [ ]:
print(f"\n{dst_alias} Contents:")
list_files(transfer_client, dst_uuid, dst_path)

Transfer task creation based on user choices

In [ ]:
task_data = globus_sdk.TransferData(
    transfer_client, src_uuid, dst_uuid
)

Get additional file transfer details from user

In [ ]:
source_file = input("Enter the file name in the source path to transfer: ").strip()
destination_file = input("Enter the destination file name (with path) to transfer: ").strip()
source_path_full = src_path + source_file
destination_path_full = dst_path + destination_file

In [ ]:
task_data.add_item(source_path_full, destination_path_full)

In [ ]:
def do_submit(client):
    task_doc = client.submit_transfer(task_data)
    task_id = task_doc["task_id"]
    print(f"submitted transfer, task_id={task_id}")